# Julia's Type System

## Part 1. Type parameters

Type parameters can be completely or partially specified:

In [33]:
Array{Int}

Array{Int64,N} where N

In [34]:
[1] isa Array

true

In [35]:
Array{Int,2}

Array{Int64,2}

A type is concrete (can have instances) if
    1. it is not declared `abstract`
    2. all parameters are specified

In [36]:
[1] isa Array{Int,1}

true

In [6]:
typeof([1]).abstract

false

### Defining types with parameters

In [37]:
[1] isa Array{Int}

true

In [38]:
[1] isa Array{Number}

false

In [39]:
Int <: Number

true

As seen above, types with different *specified* type parameters are just different, and have no subtype relationship. This is called *invariance*.

In [13]:
struct GenericPoint{T<:Real}
    x::T
    y::T
end

In [41]:
GenericPoint(1,2)

GenericPoint{Int64}(1, 2)

In [42]:
GenericPoint(1.0,2.0)

GenericPoint{Float64}(1.0, 2.0)

In [43]:
GenericPoint(1,2.0)

MethodError: MethodError: no method matching GenericPoint(::Int64, ::Float64)
Closest candidates are:
  GenericPoint(::T<:Real, !Matched::T<:Real) where T<:Real at In[40]:2

In [81]:
GenericPoint{Int64} <: GenericPoint{Real}

false

In [82]:
Real.abstract

true

#### Parametric Constructors

Parametric types add a few wrinkles to the constructor story. By default, instances of parametric composite types can be constructed either with explicitly given type parameters or with type parameters implied by the types of the arguments given to the constructor. Here are some examples:

```
julia> struct Point{T<:Real}
           x::T
           y::T
       end

julia> Point(1,2) ## implicit T ##
Point{Int64}(1, 2)

julia> Point(1.0,2.5) ## implicit T ##
Point{Float64}(1.0, 2.5)

julia> Point(1,2.5) ## implicit T ##
ERROR: MethodError: no method matching Point(::Int64, ::Float64)
Closest candidates are:
  Point(::T<:Real, ::T<:Real) where T<:Real at none:2

julia> Point{Int64}(1, 2) ## explicit T ##
Point{Int64}(1, 2)

julia> Point{Int64}(1.0,2.5) ## explicit T ##
ERROR: InexactError: Int64(2.5)
Stacktrace:
[...]

julia> Point{Float64}(1.0, 2.5) ## explicit T ##
Point{Float64}(1.0, 2.5)

julia> Point{Float64}(1,2) ## explicit T ##
Point{Float64}(1.0, 2.0)
```

### Tuple types

Very similar to other DataTypes, except
    1. Have no field names, only indices
    2. `T.parameters == T.types`
    3. Are always immutable
    4. Can have any number of fields

In [45]:
T_tuple = typeof((1,2.0))

Tuple{Int64,Float64}

In [46]:
T_tuple.parameters

svec(Int64, Float64)

In [47]:
T_tuple.types  # types of "elements"

svec(Int64, Float64)

These factors conspire to make ***Tuples the only *covariant* types in Julia:***

In [60]:
Tuple{Int} <: Tuple{Number}

true

In [73]:
Complex{Int} <: Complex{Real}

false

A Tuple type is concrete iff all its field types are.

Tuple types can be abstract with respect to the number of elements. These are called variadic tuple types, or vararg types.

In [71]:
Tuple{Int,Vararg{Int}}

Tuple{Int64,Vararg{Int64,N} where N}

Note that `Vararg` refers to the tail of a tuple type, and as such is not a first-class type itself. It only makes sense inside a Tuple type. This is a bit unfortunate.

The second parameter to `Vararg` is a length, which can also be either unspecified (as above), or specified:

In [72]:
Tuple{Int,Vararg{Int,2}}

Tuple{Int64,Int64,Int64}

## Part 2. Larger type domains

### Union types

A type can be thought of as a set of possible values. A type expresses *uncertainty* about which value we have. You can do set operations on them.

In [74]:
Union{Int64,Float64}

Union{Float64, Int64}

In [75]:
1 isa Union{Int64,Float64}

true

In [76]:
Int64 <: Number

true

In [77]:
Int64 <: Union{Int64,Float64}

true

In [78]:
Union{Int,String} <: Union{Int,String,Float32}

true

In [79]:
typeintersect(Union{Int,String}, Union{Int,String,Float32})

Union{Int64, String}

Union types naturally lend themselves to missing data.

In [80]:
data = [1.1, missing, 3.2, missing, 5.7, 0.4]

6-element Array{Union{Missing, Float64},1}:
 1.1     
  missing
 3.2     
  missing
 5.7     
 0.4     

### UnionAll types

This is an *iterated union* of types.

`Array{T,1} where T<:Integer`

Means "the union of all types of the form Array{T,1} where T is a subtype of Integer".

This expresses uncertainty about the value of a parameter.

In [83]:
# this definition is in the Base library
Vector = Array{T,1} where T

Array{T,1} where T

These are used to express "unspecified parameters".

These also describe methods with "method parameters" (or "static parameters"):

Let's first remember a simple function definition statement:

In [92]:
func_simple(a) = typeof(a)

func_simple (generic function with 1 method)

In [90]:
func_simple([0x00])

Array{UInt8,1}

Now, with an iterated union of types

In [102]:
func(a::Array{T,1}) where {T<:Integer} = T 

func (generic function with 2 methods)

In [103]:
func([0x00])

UInt8

In [104]:
func([1.0])

MethodError: MethodError: no method matching func(::Array{Float64,1})
Closest candidates are:
  func(!Matched::Array{Complex{Int64},1}) at In[91]:1
  func(!Matched::Array{T<:Integer,1}) where T<:Integer at In[102]:1

#### Question

What is the difference between

`Vector{Vector{T}} where T`

and

`Vector{Vector{T} where T}`?

Is one a subtype of the other?

In [105]:
myVector1 = Vector{Vector{T}} where T

Array{Array{T,1},1} where T

In [106]:
myVector2 = Vector{Vector{T} where T}

Array{Array{T,1} where T,1}

In the first type, all the inner vectors have the same type of elements.  In the second type, different inner vectors can have different types of elements.  An example of the first type is ``[ [1,2], [2,3], [3,4,5] ]``.  And example of the second type is ``[ [1,2], [2.0,3.0], [3,4,5] ]``. 

In [107]:
myVector2 <: myVector1

false

In [108]:
myVector1 <: myVector2

false

### Exercise

Define a `UnitPoint{<:Real}` parametric struct type which has `x` and `y` fields of type `T` and which has an inner constructor that normalizes its arguments by diving them by `hypot(x, y)` upon construction, guaranteeing that the resulting point is on the unit circle.

In [1]:
struct UnitPoint2{T<:Real}
    
    x :: T
    y :: T
    
    function UnitPoint2(x::G, y::G) where G<:Real
        
        x = x / hypot(x,y)
        y = y / hypot(x,y)
        
        new{G}(x,y)    # Note new{G}!!!
        
    end
    
end

function hypot(x,y)
    return sqrt(x^2+y^2)
end

hypot (generic function with 1 method)

In [2]:
methods(UnitPoint2)

# 1 method for generic function "(::Type)":
[1] (::Type{UnitPoint2})(x::G, y::G) where G<:Real in Main at In[1]:8

In [3]:
UnitPoint2(2.0,3.0)

UnitPoint2{Float64}(0.5547001962252291, 0.9833321660356334)

In [4]:
UnitPoint2(2,3)

InexactError: InexactError: Int64(0.5547001962252291)

Just for fun, let's see what happens if G is a supertype of T (not the same or sub-type of T).

In [7]:
struct UnitPoint3{T<:Real}
    
    x :: T
    y :: T
    
    function UnitPoint3(x::G, y::G) where G<:Number
        
        x = x / hypot(x,y)
        y = y / hypot(x,y)
        
        new{G}(x,y)
        
    end
    
end

function hypot(x,y)
    return sqrt(x^2+y^2)
end


hypot (generic function with 1 method)

In [8]:
methods(UnitPoint3)

# 1 method for type constructor:
[1] (::Type{UnitPoint3})(x::G, y::G) where G<:Number in Main at In[7]:8

In [9]:
UnitPoint3(2.0,3.0)  # Julia automatically figures out the arguments are Float64, so we don't need to call UnitPoint3{xxx}

UnitPoint3{Float64}(0.5547001962252291, 0.9833321660356334)

In [10]:
subtypes(Number)

2-element Array{Any,1}:
 Complex
 Real

In [11]:
UnitPoint3(2.0+2.0im,3.0+3.0im)

TypeError: TypeError: in UnitPoint3, in T, expected T<:Real, got Type{Complex{Float64}}

Now, if we define an inner constructor explicitly with a type parameter:

In [12]:
struct UnitPoint{T<:Real}
    
    x :: T
    y :: T
    
    function UnitPoint{G}(x::G, y::G) where G<:Real
        
        x = x / hypot(x,y)
        y = y / hypot(x,y)
        
        new(x,y)
        
    end
    
end

function hypot(x,y)
    return sqrt(x^2+y^2)
end

hypot (generic function with 1 method)

In [13]:
methods(UnitPoint)

# 0 methods for type constructor:

In [14]:
UnitPoint(2.0,3.0)

MethodError: MethodError: no method matching UnitPoint(::Float64, ::Float64)

***We have to use UnitPoint{xxx} because the inner constructor explicitly has a type parameter:***

In [15]:
methods(UnitPoint{Float64})

# 1 method for type constructor:
[1] (::Type{UnitPoint{G}})(x::G, y::G) where G<:Real in Main at In[12]:8

In [16]:
UnitPoint{Float64}(2.0,3.0)

UnitPoint{Float64}(0.5547001962252291, 0.9833321660356334)

### One More Exercise

In [15]:
struct Point{T<:Real}
    x::T
    y::T

    Point{G}(x,y) where {G<:Integer} = new(55,y)
    Point{H}(x,y) where {H<:AbstractFloat} = new(x, 11)
end

Point(x::T, y::T) where {T<:Integer} = Point{T}(x,y)
Point(x::T, y::T) where {T<:AbstractFloat} = Point{T}(x,y)
println(methods(Point))

# 2 methods for generic function "(::Type)":
[1] (::Type{Point})(x::T, y::T) where T<:Integer in Main at In[15]:9
[2] (::Type{Point})(x::T, y::T) where T<:AbstractFloat in Main at In[15]:10


In [16]:
p = Point(2,3)

Point{Int64}(55, 3)

In [17]:
p2 = Point(2.0,3.0)

Point{Float64}(2.0, 11.0)

**We do not suffer from infinite recursion even though the outer constructor appears to call itself!!**  (See Exercise 17-2)

This is because the inner constructor is called with a type parameter {T} by the outer constructor as opposed to the outer constructor itself.